## Import Lib

In [ ]:
import pandas as pd
from datetime import timedelta
import os
import numpy as np

In [ ]:
path = r"C:\Users\Akhil\Downloads\project\TradeX_raw\STcon"


os.chdir(path)
print("Current Working Directory:", os.getcwd())


xlsx_files = [f for f in os.listdir(path) if f.endswith('.xlsx')]

if len(xlsx_files) != 3:
    raise ValueError("There should be exactly 3 .xlsx files in the directory.")

In [ ]:

st_1 = pd.read_excel(xlsx_files[0])
st_2 = pd.read_excel(xlsx_files[1])
st_3 = pd.read_excel(xlsx_files[2])


concat_df = pd.concat([st_1,st_2,st_3], ignore_index=True)

print("Length of st_1:", len(st_1))
print("Length of st_2:", len(st_2))
print("Length of st_3:", len(st_3))
print("Length of concatenated DataFrame:", len(concat_df))

# A = concat_df[concat_df['End of Call code'] != 'CAN_NOT_MAKE_CALL']

output_path = r"C:\Users\Akhil\Downloads\project\TradeX_raw\S_29_09.csv"
concat_df.to_csv(output_path, index=False)
print(f"CSV saved to: {output_path}")

In [ ]:
st_3.dtypes

In [ ]:
path = r"C:\Users\Akhil\Downloads\project\TradeX_raw"

#\pyton automation\Tradex_dialer_raw
# Change the working directory
os.chdir(path)

# Confirm the new working directory
print("Current Working Directory:", os.getcwd())

## load csv file

In [ ]:
# voiso = pd.read_csv('VT_12-16.csv',low_memory=False)
tata = pd.read_csv('T_29_09.csv',low_memory=False)
know = pd.read_csv('K_29_09.csv',low_memory=False)
Qconn = pd.read_csv('Q_18_07.csv',low_memory=False)
stringee = pd.read_csv('S_29_09.csv',low_memory=False)

## Check_point_1

In [ ]:
# print("voiso:", voiso['Date and time'].unique()[:5])
print("tata:", tata['Call Start Date'].unique()[:5])
print("know:", know['Date and Time'].unique()[:5])
print("Qconn:", Qconn['Date time'].unique()[:5])
print("stringee:", stringee['Start time'].unique()[:5])

In [ ]:
stringee

In [ ]:
stringee['Start time'] = pd.to_datetime(stringee['Start time']) + pd.Timedelta(hours=1, minutes=30)

In [ ]:
stringee.columns

In [ ]:
stringee['Answer duration'].dtype

In [ ]:
# Function to convert durations in hh:mm:ss format to timedelta
def duration_to_timedelta(duration):
    hours, minutes, seconds = map(int, duration.split(':'))
    return timedelta(hours=hours, minutes=minutes, seconds=seconds)

# Convert the 'Queue duration' and 'Answer duration' columns to timedelta
stringee['Queue Duration (timedelta)'] = stringee['Queue duration'].apply(duration_to_timedelta)
stringee['Answer Duration (timedelta)'] = stringee['Answer duration'].apply(duration_to_timedelta)

# Calculate the total duration as timedelta
stringee['Total Duration (timedelta)'] = stringee['Queue Duration (timedelta)'] + stringee['Answer Duration (timedelta)']

# Convert the total duration back to hh:mm:ss format (remove "0 days")
stringee['Total Duration'] = stringee['Total Duration (timedelta)'].apply(lambda x: str(x).split(", ")[-1])

# Drop intermediate timedelta columns if not needed
stringee = stringee.drop(columns=['Queue Duration (timedelta)', 'Answer Duration (timedelta)', 'Total Duration (timedelta)'])


## ETL

In [ ]:
new_tata = tata[['Call Start Date', 'Connected to Agent','Call Status','Answer Duration (HH:MM:SS)',
                 'Hold Duration (HH:MM:SS)','Total Call Duration (HH:MM:SS)','Call Start Time','Customer Number']]

In [ ]:

new_Know = know[['Date and Time', 'Agent Name','Call Status', 'Talk Time (hh:mm:ss)', 'Hold Time (hh:mm:ss)','Total Call Duration (hh:mm:ss)','Customer']]

In [ ]:
# new_voiso =  voiso[['Date and time','Agent(s)','Disposition','Talk time','Duration','DNIS/To']]

In [ ]:
new_qconn = Qconn[['Date time','Agent Mobile','Call Event','Transfer Duration','Duration','User Mobile']]

In [ ]:
new_string = stringee[['Start time','Account','Call status','Answer duration','Hold duration','Total Duration','Customer number']]

## Voiso date 

In [ ]:
# new_voiso['Date and time'].unique()

In [ ]:
# # Function to convert 'Date and time' from mm/dd/yyyy HH:mm:ss to yyyy/mm/dd HH:mm:ss
# def fix_voiso_datetime(df, col_name):
#     # Convert using pd.to_datetime, specify the format as mm/dd/yyyy
#     df[col_name] = pd.to_datetime(df[col_name], errors='coerce', format='%m/%d/%Y %H:%M:%S') #
    
#     # Format the datetime to 'yyyy/mm/dd HH:mm:ss'
#     df[col_name] = df[col_name].dt.strftime('%Y/%m/%d %H:%M:%S')
    
#     return df

# # Apply the function to 'Date and time' column in Voiso
# new_voiso = fix_voiso_datetime(new_voiso, 'Date and time')

# # Verify the result
# print("Voiso Date and time after fixing format:", new_voiso['Date and time'].unique())


## string date format

In [ ]:
#####check
new_string['Start time'].head()

In [ ]:
# Function to fix 'new_string' Date Time format
def fix_string_datetime(date_str):
    try:
        # Parse the date and time, assuming format mm/dd/yyyy hh:mm:ss AM/PM
        parsed_date = pd.to_datetime(date_str, errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')
        return parsed_date
    except Exception as e:
        return pd.NaT  # Return NaT for any unparseable date

# Apply this function to the 'Date and Time' column in 'new_string'
new_string['Start time'] = new_string['Start time'].apply(fix_string_datetime)

# Now format it to yyyy/mm/dd HH:mm:ss
new_string['Start time'] = new_string['Start time'].dt.strftime('%Y/%m/%d %H:%M:%S')

# Check the result
print("String unique date formats after fixing:", new_string['Start time'].head())

In [ ]:
new_Know['Date and Time'].head()


In [ ]:
new_qconn['Date time'].head()

In [ ]:
# Convert 'Date and Time' column in new_Know to datetime format
new_qconn['Date time'] = pd.to_datetime(new_qconn['Date time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
new_Know['Date and Time'] = pd.to_datetime(new_Know['Date and Time'], errors='coerce', format='%Y-%m-%d %H:%M:%S') #%d/%m/%Y

# Now check the type of 'Date' again in both dataframes
print("Data type of 'Date' in new_qconn after conversion:", new_qconn['Date time'].dtype)
print("Data type of 'Date' in new_Know after conversion:", new_Know['Date and Time'].dtype)

# Extract date and call start time for new_Know
new_Know['Date'] = new_Know['Date and Time'].dt.date  # Extract date part
new_Know['Call Start Time'] = new_Know['Date and Time'].dt.strftime('%H:%M:%S')  # Extract time part

# Extract date and call start time for new_qconn (if you haven't done this yet)
new_qconn['Date'] = new_qconn['Date time'].dt.date  # Extract date part
new_qconn['Call Start Time'] = new_qconn['Date time'].dt.strftime('%H:%M:%S')  # Extract time part

# Check the resulting dataframes to ensure extraction is successful
print(new_qconn[['Date', 'Call Start Time']].head())
print(new_Know[['Date', 'Call Start Time']].head())


In [ ]:
# Find rows where 'Date time' is NaT
invalid_dates = new_qconn[new_qconn['Date time'].isna()]
print(invalid_dates)


In [ ]:
(new_qconn['Date'].unique())

In [ ]:
# Check the data type of the 'Date' column
print("Data type of 'Date' in new_qconn:", new_qconn['Date time'].dtype)
print("Data type of 'Date' in new_Know:", new_Know['Date and Time'].dtype)


In [ ]:
# Function to separate Date and Call Start Time
def split_date_time(df, col_name):
    # Convert to datetime first (if not already done)
    df[col_name] = pd.to_datetime(df[col_name], errors='coerce', format='%Y/%m/%d %H:%M:%S')

    # Extract the Date (yyyy/mm/dd) and Time (hh:mm:ss)
    df['Date'] = df[col_name].dt.date
    df['Call Start Time'] = df[col_name].dt.strftime('%H:%M:%S')

    return df

# Apply the function to each dataframe
# new_voiso = split_date_time(new_voiso, 'Date and time')
new_string = split_date_time(new_string, 'Start time')

# Verify the changes
print(new_qconn[['Date', 'Call Start Time']].head())
# print(new_voiso[['Date', 'Call Start Time']].head())
print(new_Know[['Date', 'Call Start Time']].head())
print(new_string[['Date', 'Call Start Time']].head())


In [ ]:
new_qconn

### copy of main dfs

In [ ]:
tata_copy = new_tata.copy()
know_copy = new_Know.copy()
# voiso_copy = new_voiso.copy()
qconn_copy = new_qconn.copy()
string_copy = new_string.copy()

### insert source 

In [ ]:
tata_copy['Source'] = 'Tata'
know_copy['Source'] = 'Knowlarity'
# voiso_copy['Source'] = 'Voiso'
qconn_copy['Source'] = 'Qkonnect'
string_copy['Source'] = 'Stringee'


In [ ]:
print(tata_copy.dtypes)
print(string_copy['Date'].unique())

### Rename Columns

In [ ]:
tata_copy.rename(columns={
    'Call Start Date': 'Date',
    'Connected to Agent': 'Dialer Name',
    'Customer Number' : 'Number',
    'Call Status': 'Call Status',
    'Answer Duration (HH:MM:SS)': 'Talk Time',
    'Hold Duration (HH:MM:SS)': 'Hold Time',
    'Total Call Duration (HH:MM:SS)': 'Total Call Duration',
    'Call Start Time': 'Call Start Time'
}, inplace=True)

know_copy.rename(columns={
    'Date': 'Date',
    'Agent Name': 'Dialer Name',
    'Customer': 'Number',
    'Call Status': 'Call Status',
    'Talk Time (hh:mm:ss)': 'Talk Time',
    'Hold Time (hh:mm:ss)': 'Hold Time',
    'Total Call Duration (hh:mm:ss)': 'Total Call Duration',
    'Call Start Time': 'Call Start Time'
}, inplace=True)

# voiso_copy.rename(columns={
#     'Date': 'Date',
#     'Agent(s)': 'Dialer Name',
#     'DNIS/To': 'Number',
#     'Disposition': 'Call Status',
#     'Talk time': 'Talk Time',
#     'Duration': 'Total Call Duration',
#     'Call Start Time': 'Call Start Time'
# }, inplace=True)

qconn_copy.rename(columns={
    'Date': 'Date',
    'Agent Mobile': 'Dialer Name',
    'User Mobile': 'Number',
    'Call Event': 'Call Status',
    'Transfer Duration': 'Talk Time',
    'Duration': 'Total Call Duration',
    'Call Start Time': 'Call Start Time'
}, inplace=True)

string_copy.rename(columns={
    'Date': 'Date',    
    'Account': 'Dialer Name',
    'Customer number': 'Number',
    'Call status': 'Call Status',
    'Answer duration': 'Talk Time',
    'Hold duration': 'Hold Time',
    'Total Duration': 'Total Call Duration',
    'Call Start Time': 'Call Start Time'
}, inplace=True)



In [ ]:
print(qconn_copy.shape)
# print(voiso_copy.shape)
print(know_copy.shape)  
print(tata_copy.shape)
print(string_copy.shape)

In [ ]:
print(string_copy['Date'].unique())
print(tata_copy['Date'].unique())
print(know_copy['Date'].unique())
# print(voiso_copy['Date'].unique())
print(qconn_copy['Date'].unique())


In [ ]:
tata_copy.shape, know_copy.shape, qconn_copy.shape,string_copy.shape

In [ ]:

# Select only the required columns from each dataframe
tata_selected = tata_copy[[ 'Source','Date', 'Dialer Name','Number', 'Call Status','Call Start Time','Total Call Duration', 'Talk Time', 'Hold Time']]
know_selected = know_copy[[ 'Source','Date', 'Dialer Name','Number' ,'Call Status', 'Call Start Time','Total Call Duration','Talk Time', 'Hold Time']]
# voiso_selected = voiso_copy[[ 'Source','Date', 'Dialer Name', 'Number','Call Status','Call Start Time','Total Call Duration', 'Talk Time']]
qconn_selected = qconn_copy[['Source','Date', 'Dialer Name','Number', 'Call Status','Call Start Time','Total Call Duration', 'Talk Time']]
string_selected = string_copy[['Source','Date', 'Dialer Name','Number', 'Call Status','Call Start Time','Total Call Duration', 'Talk Time', 'Hold Time']]


# Now concatenate
combined = pd.concat([tata_selected, know_selected, qconn_selected, string_selected], ignore_index=True)

In [ ]:
combined

In [ ]:
print(combined.isnull().sum())
print(combined.shape)

## Check_point_2

In [ ]:
# Filter rows where Date is null
null_date_entries = combined[combined['Date'].isnull()]

# Display the filtered DataFrame
null_date_entries


In [ ]:
combined['Date'].unique()

In [ ]:
combined_df =combined.copy()

In [ ]:
# Refine the regex to only remove unwanted patterns
combined_df['Dialer Name'] = combined_df['Dialer Name'].str.replace(r"\s*\([^)]*\)|@.*|;.*", "", regex=True)

# Fill missing 'Dialer Name' values with their original values if they were numeric
combined_df['Dialer Name'] = combined_df['Dialer Name'].fillna(combined['Dialer Name'])

In [ ]:
# Replace NaN in 'Hold time' with '00:00:00'
combined_df['Hold Time'] = combined_df['Hold Time'].fillna('00:00:00')

combined_df

In [ ]:
A = combined_df.copy()

In [ ]:
# Remove rows where 'Dialer Name' is null
A1 = A[A['Dialer Name'].notnull()]

# Verify the result
print(f"Number of rows after removing null 'Dialer Name': {len(A1)}")

In [ ]:
A1.describe()

In [ ]:
# Check for null values in the 'Dialer Name' column
null_dialer_name_count = combined_df['Dialer Name'].isnull().sum()

# Print the result
print(f"Number of null values in 'Dialer Name': {null_dialer_name_count}")

In [ ]:
A1

## Check_point_3

In [ ]:
unique_dates_per_source = A1.groupby('Source')['Date'].unique()
print(unique_dates_per_source)

In [ ]:
A1

In [ ]:
import re

def normalize_talk_time(talk_time):
    # If the value is in seconds (only digits), convert it to hh:mm:ss
    if re.match(r"^\d+$", str(talk_time)):
        seconds = int(talk_time)
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    elif re.match(r"^\d+:\d+:\d+$", str(talk_time)):
        parts = talk_time.split(":")
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    else:
        # Return the value as-is if it doesn't match expected formats
        return talk_time

# Apply the normalization function to the 'Talk Time' column
A1['Talk Time'] = A1['Talk Time'].apply(normalize_talk_time)

# Apply the normalization function to the 'Total Call Duration' column
A1['Total Call Duration'] = A1['Total Call Duration'].apply(normalize_talk_time)


print(A1[['Talk Time', 'Total Call Duration']].head())

In [ ]:
unique_talk_time_formats = A1['Talk Time'].unique()

print(f"Unique formats in 'Talk Time': {unique_talk_time_formats}")

#### Unique call status (AOI)

In [ ]:
A1['Call Status'].unique()

In [ ]:
A1['Call Status'] = A1['Call Status'].apply(
    lambda x: 'connected' if str(x).lower() == 'answered' else 'not connected'
)

In [ ]:
A1

In [ ]:
len(A1['Dialer Name'].unique())

In [ ]:
A1['Dialer Name'].unique()

In [ ]:
# Count the occurrences of each unique 'Dialer Name' and sort from highest to lowest
dialer_name_counts = A1['Dialer Name'].value_counts().sort_values(ascending=False)

print(dialer_name_counts)

In [ ]:

combined_df_1 = A1[~A1['Dialer Name'].isin([None, '---'])]

combined_df_1.reset_index(drop=True, inplace=True)

combined_df_1

In [ ]:
combined_df_1

In [ ]:
ref_d = pd.read_csv('Team_tradex.csv')

In [ ]:
ref_d.nunique()

In [ ]:
ref= ref_d.copy()

In [ ]:
ref = ref[~ref['Email'].str.contains('inactive', case=False, na=False)]

In [ ]:
ref

## Checkpoint_4

In [ ]:
ref.nunique()

In [ ]:
for df in [ref, combined_df_1]:
    df['Dialer Name'] = (
        df['Dialer Name']
        .astype(str)  # Convert all values to strings
        .str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with a single space
        .str.strip()  # Remove leading and trailing spaces
        .str.replace(r'@.*', '', regex=True)  # Remove everything from and after '@'
        .str.replace(r'\(.*', '', regex=True)  # Remove everything from and after '('
        .str.strip()  # Remove any trailing spaces left after replacements
        .str.lower()  # Convert all text to lowercase
    )


In [ ]:
ref.shape

In [ ]:
ref.isnull().sum()

In [ ]:
dup = ref[ref.duplicated(subset=['Dialer Name','Email','Dialer'], keep=False)]
dup

In [ ]:
ref = ref.drop_duplicates(subset=['Dialer Name','Email','Dialer'])

In [ ]:
ref.rename(columns={'Email': 'CRM ID'}, inplace=True)

In [ ]:
ref

In [ ]:
# Ensure 'Dialer Name' in both dataframes is treated as a string
combined_df_1['Dialer Name'] = combined_df_1['Dialer Name'].astype(str)
ref['Dialer Name'] = ref['Dialer Name'].astype(str)

# Merge the dataframes on 'Dialer Name'
combined_df_2 = combined_df_1.merge(ref, how='left', left_on='Dialer Name', right_on='Dialer Name')


combined_df_2


In [ ]:
# Filter rows where 'CRM ID' is null
crm_id_null_df = combined_df_2[combined_df_2['CRM ID'].isnull()]


In [ ]:

crm_id_null_df['Dialer Name'].unique()

In [ ]:
crm_id_null_df

## check_point_5

In [ ]:
source = crm_id_null_df.groupby('Source')['Dialer Name'].unique()
source

In [ ]:
crm_id_null_df.shape

## Raw data

In [ ]:
combined_df_2

In [ ]:
Dialers = combined_df_2[combined_df_2['CRM ID'].notnull() & combined_df_2['Talk Time'].notnull()].copy()

In [ ]:
Dialers.isnull().sum()

In [ ]:
Dialers = Dialers.drop_duplicates(subset=['Number','Call Start Time'])


In [ ]:
XXX = combined_df_2[combined_df_2['Date'].isnull()]
XXX

In [ ]:
Dialers.dtypes

In [ ]:
from datetime import timedelta

Dialers['Date'] = pd.to_datetime(Dialers['Date'], format='%Y-%m-%d', errors='coerce')
Dialers['Call Start Time'] = pd.to_datetime(
    Dialers['Date'].dt.strftime('%Y-%m-%d').fillna('1900-01-01') + ' ' + Dialers['Call Start Time'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

# Convert durations
Dialers['Total Call Duration'] = Dialers['Total Call Duration'].apply(
    lambda x: pd.to_timedelta(x) if isinstance(x, str) else pd.Timedelta(0)
)

# Sort
Dialers = Dialers.sort_values(by=['Date', 'CRM ID', 'Call Start Time']).reset_index(drop=True)

# Init columns
Dialers['Call Gap'] = 'No'
Dialers['Gap Duration'] = '00:00:00'

# Define working hours
start_time = pd.to_datetime('09:30:00').time()
end_time = pd.to_datetime('18:30:00').time()

# Loop through to calculate gaps
for i in range(1, len(Dialers)):
    same_crm = Dialers.loc[i, 'CRM ID'] == Dialers.loc[i - 1, 'CRM ID']
    same_date = Dialers.loc[i, 'Date'] == Dialers.loc[i - 1, 'Date']  

    if same_crm and same_date:
        current_time = Dialers.loc[i, 'Call Start Time'].time()
        previous_time = Dialers.loc[i - 1, 'Call Start Time'].time()

        previous_end = Dialers.loc[i - 1, 'Call Start Time'] + Dialers.loc[i - 1, 'Total Call Duration']
        gap_duration = Dialers.loc[i, 'Call Start Time'] - previous_end

        if gap_duration.total_seconds() < 0:
            gap_duration = timedelta(0)

        # Store formatted gap
        total_seconds = int(gap_duration.total_seconds())
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60
        Dialers.loc[i, 'Gap Duration'] = f"{hours:02}:{minutes:02}:{seconds:02}"

        # Set 'Call Gap' only if within working hours
        if start_time <= current_time <= end_time and start_time <= previous_time <= end_time:
            Dialers.loc[i, 'Call Gap'] = 'Yes' if gap_duration > timedelta(minutes=1) else 'No'

Dialers



In [ ]:
Dialers.isnull().sum()

In [ ]:
import re
invalid_values = []

# Function to convert mixed formats to seconds
def to_seconds(value):
    try:
        if isinstance(value, pd.Timedelta):
            return int(value.total_seconds())  # Convert timedelta to seconds
        elif re.match(r"^\d{1,2}:\d{1,2}:\d{1,2}$", str(value)):
            parts = list(map(int, value.split(':')))
            while len(parts) < 3:
                parts.insert(0, 0)  
            return parts[0] * 3600 + parts[1] * 60 + parts[2]
        elif str(value).isdigit():
            return int(value)
        else:
            invalid_values.append(value)
            return value
    except Exception:
        invalid_values.append(value)
        return value

# Replace missing values with empty strings
Dialers['Talk Time'] = Dialers['Talk Time'].fillna('')
Dialers['Hold Time'] = Dialers['Hold Time'].fillna('')
Dialers['Total Call Duration'] = Dialers['Total Call Duration'].fillna('')
Dialers['Gap Duration'] = Dialers['Gap Duration'].fillna('')

# Convert Talk Time, Hold Time, and Total Call Duration to seconds
Dialers['Talk Time (seconds)'] = Dialers['Talk Time'].apply(to_seconds)
Dialers['Hold Time (seconds)'] = Dialers['Hold Time'].apply(to_seconds)
Dialers['Total Duration (seconds)'] = Dialers['Total Call Duration'].apply(to_seconds)
Dialers['Gap Duration (seconds)'] = Dialers['Gap Duration'].apply(to_seconds)

# Notify invalid values
if invalid_values:
    print("Invalid values found in 'Talk Time', 'Total Call Duration', or 'Hold Time':, or 'Call Gap Duration':")
    print(invalid_values)

In [ ]:
Dialers.isnull().sum()

### Change


In [ ]:
# df['Total_Duration'] = pd.to_timedelta(df['Total_Duration'], errors='coerce')

In [ ]:
# Group by 'CRM ID' and 'Date' and calculate 
A = Dialers.groupby(['CRM ID', 'Date']).agg(
    Total_Dialed_Calls=('Call Status', 'count'),
    Unique_Dialed_Numbers=('Number', 'nunique'),
    Total_Connected_Calls=('Call Status', lambda x: (x == 'connected').sum()),
    Total_Number_of_Call_Gap=('Call Gap', lambda x: (x == 'Yes').sum()),
    Total_Call_GT_30=('Talk Time (seconds)', lambda x: ((Dialers.loc[x.index, 'Call Status'] == 'connected') & (x > 30)).sum()),
    Total_Duration=('Total Duration (seconds)', 'sum'),
    Total_Talk_Time=('Talk Time (seconds)', lambda x: x[Dialers.loc[x.index, 'Call Status'] == 'connected'].sum()),
    Total_Talk_Time_GT_30=('Talk Time (seconds)', lambda x: x[(Dialers.loc[x.index, 'Call Status'] == 'connected') & (x > 30)].sum()),
    Total_Connected_Hold_Time=('Hold Time (seconds)', lambda x: x[Dialers.loc[x.index, 'Call Status'] == 'connected'].sum()), 
    Total_Gap_Duration=('Gap Duration (seconds)', 'sum')
).reset_index()

# Fix: Subtract 1hr only if greater, else keep the original value
A['Total_Gap_Duration'] = A['Total_Gap_Duration'].apply(lambda x: x - 3600 if x > 3600 else x)


A['Avg_Gap_per_call'] = A['Total_Gap_Duration'] / A['Total_Dialed_Calls']

# New: Gap Duration After Leverage (give 45 seconds per call, subtract from actual gap used, add to 0 if negative)
A['Gap Duration After Leverage'] = (A['Total_Gap_Duration'] - (A['Total_Dialed_Calls'] * 45)).clip(lower=0)

# Recalculate Login Hours after updated gap
A['Login Hours'] = A['Total_Duration'] + A['Total_Gap_Duration'] + 3600
A['Login Hours'] = pd.to_timedelta(A['Login Hours'], unit='s')
A['Login Hours'] = A['Login Hours'].apply(lambda x: str(x).split()[-1])

# Convert 'Login Hours' string (hh:mm:ss) to timedelta
A['Login Hours (Timedelta)'] = pd.to_timedelta(A['Login Hours'])

# Defining attendance logic
def mark_attendance(td):
    if td < pd.Timedelta(hours=4, minutes=30):
        return 'Absent'
    elif td < pd.Timedelta(hours=6):
        return 'Half Day'
    elif td < pd.Timedelta(hours=8, minutes=30):
        return 'Warning'
    else:
        return 'Present'


A['Attendance'] = A['Login Hours (Timedelta)'].apply(mark_attendance)


A.drop(columns='Login Hours (Timedelta)', inplace=True)


A


In [ ]:
AB = A['Attendance'].unique()
AB

In [ ]:
ref

In [ ]:
# Filter unique CRM ID and select specific columns
unique_crm_ref = ref.drop_duplicates(subset=['CRM ID'])[['CRM ID','Employee code', 'Full Name','Pool', 'TL','Vertical']]

unique_crm_ref

In [ ]:
# Extract unique dates from DataFrame A
all_dates = A['Date'].unique()

# Create a DataFrame with all CRM IDs from unique_crm_ref and all dates
date_crm_combinations = pd.MultiIndex.from_product(
    [unique_crm_ref['CRM ID'], all_dates],
    names=['CRM ID', 'Date']
).to_frame(index=False)



merged = date_crm_combinations.merge(
    A,
    how='left',
    on=['CRM ID', 'Date']
).fillna(0)  


In [ ]:
merged

In [ ]:
# Outer-merge with unique_crm_ref to retain all CRM IDs
merged_df = unique_crm_ref.merge(
    merged,
    how='outer',
    on='CRM ID'
)

In [ ]:
merged_df

In [ ]:
# Replace NaN or missing values with 0
columns_to_format = ['Total_Dialed_Calls', 'Unique_Dialed_Numbers','Total_Connected_Calls','Total_Number_of_Call_Gap', 'Total_Call_GT_30','Total_Duration','Total_Talk_Time','Total_Talk_Time_GT_30','Total_Connected_Hold_Time','Total_Gap_Duration']
merged_df[columns_to_format] = merged_df[columns_to_format].fillna(0)

# Convert specified columns to integers
merged_df[columns_to_format] = merged_df[columns_to_format].astype(int)

# Reorder columns
formatted_df = merged_df[['Date', 'Pool', 'TL', 'CRM ID','Employee code', 'Full Name', 'Vertical'] + [col for col in merged_df.columns if col not in ['Date', 'Pool', 'TL', 'CRM ID','Employee code', 'Full Name', 'Vertical']]]

formatted_df

In [ ]:
formatted_df.isnull().sum()

In [ ]:
def seconds_to_hhmmss(seconds):
    if pd.isna(seconds):
        return None  # or "00:00:00" if you prefer
    total_seconds = round(seconds)
    hours = total_seconds // 3600
    remaining = total_seconds % 3600
    minutes = remaining // 60
    seconds = remaining % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"  # hh:mm:ss


columns_to_convert = [
    'Total_Duration', 
    'Total_Talk_Time', 
    'Total_Talk_Time_GT_30', 
    'Total_Connected_Hold_Time', 
    'Total_Gap_Duration',
    'Avg_Gap_per_call',
    'Gap Duration After Leverage'
]

for col in columns_to_convert:
    try:
        formatted_df[col] = formatted_df[col].apply(seconds_to_hhmmss)
    except Exception as e:
        print(f"Error in column: {col}")
        raise e  # re-raise the error so you still get the traceback


In [ ]:
# from datetime import timedelta

# def seconds_to_hhmmss(seconds):
#     total_seconds = round(seconds)
#     hours = total_seconds // 3600
#     remaining = total_seconds % 3600
#     minutes = remaining // 60
#     seconds = remaining % 60
#     return f"{hours:02}:{minutes:02}:{seconds:02}"  # hh:mm:ss


# columns_to_convert = [
#     'Total_Duration', 
#     'Total_Talk_Time', 
#     'Total_Talk_Time_GT_30', 
#     'Total_Connected_Hold_Time', 
#     'Total_Gap_Duration',
#     'Avg_Gap_per_call',
#     'Gap Duration After Leverage'
# ]

# for col in columns_to_convert:
#     formatted_df[col] = formatted_df[col].apply(seconds_to_hhmmss)	


In [ ]:
formatted_df = formatted_df.sort_values(by=['Date','CRM ID'])

In [ ]:
formatted_df

In [ ]:
formatted_df['Attendance'].unique()

In [ ]:

formatted_df['Login Hours'] = formatted_df['Login Hours'].apply(lambda x: '00:00:00' if x == 0 else x)

formatted_df['Attendance'] = formatted_df['Attendance'].apply(lambda x: 'Absent' if x == 0 else x)


In [ ]:
formatted_df

In [ ]:
# Convert 'Total Call Duration' to hh:mm:ss format
Dialers["Total Call Duration"] = Dialers["Total Call Duration"].apply(lambda x: str(x).split(" ")[-1])
Dialers['Number'] = "'" + Dialers['Number'].astype(str)


In [ ]:
D = Dialers.drop(columns=['Talk Time (seconds)','Hold Time (seconds)','Total Duration (seconds)','Gap Duration (seconds)','Dialer'])
D['Number'] = D['Number'].astype(str).str.split('.').str[0]
D

In [ ]:
D.isnull().sum()

# Transaction_stich

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import itables
itables.init_notebook_mode(all_interactive=True)
from itables import show
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = D.copy()

In [ ]:
unique_calls = df.drop_duplicates(subset=["Date", "CRM ID", "Number","Call Status"])[["Date", "CRM ID", "Number","Call Status","Pool","TL","Full Name"]]

In [ ]:

# Function to clean phone numbers
def clean_phone(phone):
    if pd.isna(phone):
        return ""
    cleaned = ''.join(filter(str.isdigit, str(phone)))
    return cleaned[-10:]


unique_calls["Consider"] = unique_calls["Number"].apply(clean_phone)


unique_calls["Len"] = unique_calls["Consider"].str.len()


In [ ]:
unique_calls

# Trnx_Db

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta
import warnings
 
warnings.filterwarnings('ignore')
 
import numpy as np
import itables
itables.init_notebook_mode(all_interactive=True)

In [ ]:
D.isnull().sum()

In [ ]:
# Your connection parameters
conn_params = {
    'host': 'your_host',
    'user': 'your_user',
    'password': 'your_password',
    'database': 'your_database'
}


D['Date'] = pd.to_datetime(D['Date'])
dates_only = D['Date'].dt.date


overall_start_date = datetime.combine(dates_only.min(), datetime.min.time())
 
overall_end_date = datetime.combine(dates_only.max(), datetime.min.time()) + timedelta(days=1)
 
print(f"Data fetch range from {overall_start_date} to {overall_end_date}")
 
# Initialize empty dataframe and loaded_dates set if not already
try:
    partial_df
except NameError:
    partial_df = pd.DataFrame()
 
try:
    loaded_dates
except NameError:
    loaded_dates = set()


# Function to fetch data chunk for a date range from MySQL
def fetch_data_for_date_range(start_date, end_date):
    query_template = """
    SELECT *
    FROM manual_transaction_log
    WHERE time >= %s AND time < %s
      AND action NOT IN ('Delete', 'Update')
      AND type NOT IN ('Pending', 'Withdraw Request', 'Deposit Request')
      AND typeDetail NOT IN ('SLTP', 'Adjustment')
    ORDER BY time
    """
    conn = mysql.connector.connect(**conn_params)
    df = pd.read_sql(query_template, conn, params=(start_date, end_date))
    conn.close()
    return df


# VPN Checkpoint

In [ ]:
current_start = overall_start_date
if not loaded_dates:
    print("Starting fresh from", current_start.strftime('%Y-%m-%d'))
else:
    # Resume from last loaded date + 1 day
    last_loaded = max(datetime.strptime(d, '%Y-%m-%d') for d in loaded_dates)
    current_start = last_loaded + timedelta(days=1)
    print("Resuming from", current_start.strftime('%Y-%m-%d'))

accumulated_rows = len(partial_df)

while current_start < overall_end_date:
    current_end = current_start + timedelta(days=1)
    current_start_str = current_start.strftime('%Y-%m-%d')

    if current_start_str in loaded_dates:
        print(f"Skipping already loaded date {current_start_str}")
        current_start = current_end
        continue

    try:
        print(f"Loading data from {current_start_str} to {current_end.strftime('%Y-%m-%d')}")

        df_chunk = fetch_data_for_date_range(current_start, current_end)
        num_rows = len(df_chunk)
        accumulated_rows += num_rows
        print(f"Loaded {num_rows} rows; Total rows loaded so far: {accumulated_rows}")

        # Append chunk to partial_df (reset index for concatenation)
        partial_df = pd.concat([partial_df, df_chunk], ignore_index=True)

        loaded_dates.add(current_start_str)

        current_start = current_end

    except Exception as e:
        print(f"Error loading data for {current_start_str}: {e}")
        print("Stop and rerun this cell to resume.")
        break  # Stop the loop to fix issue or rerun later


### final_df

In [ ]:
final_df = partial_df.copy()
print(f"Final combined data with {len(final_df)} rows is ready.")

## Consolidation

In [ ]:
combined_df = final_df.copy()  

In [ ]:
combined_df

In [ ]:
# Get all rows where ticketId and time are duplicated
duplicates = combined_df[combined_df.duplicated(subset=['ticketId', 'time','accountId'], keep=False)]

In [ ]:
# See how many
print(f"Total duplicate rows: {len(duplicates)}")

# Show unique duplicate pairs (ticketId, time) with counts
duplicate_summary = (
    duplicates.groupby(['ticketId', 'time'])
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

print(duplicate_summary)

In [ ]:
duplicates

In [ ]:
# Drop duplicates based on specific subset
final_df = combined_df.drop_duplicates(subset=['ticketId', 'time', 'accountId'], keep='first')

print(f"Rows before: {len(combined_df)}")
print(f"Rows after dropping duplicates: {len(final_df)}")

In [ ]:
check = final_df.copy()

### Min date and max date

In [ ]:
# Parse dd/mm/yyyy H:MM
parsed_ddmmyyyy = pd.to_datetime(check['time'], format="%d/%m/%Y %H:%M", errors='coerce')

# Wherever parsing succeeded, replace the original with the standardized format
check.loc[parsed_ddmmyyyy.notna(), 'time'] = parsed_ddmmyyyy.dt.strftime("%Y-%m-%d %H:%M:%S")

# Now ensure the whole column is in datetime
check['time'] = pd.to_datetime(check['time'], errors='coerce')

print(check['time'].head())


In [ ]:
check['date_only'] = check['time'].dt.date

In [ ]:

# Get min and max date
min_date = check['date_only'].min()
max_date = check['date_only'].max()
print(f"Min date: {min_date}, Max date: {max_date}")

# Create a date range for every day between min and max date
full_date_range = pd.date_range(start=min_date, end=max_date)

# Prepare DataFrame for month-wise date counts and missing dates
months = check['date_only'].apply(lambda x: x.replace(day=1)).unique()
months.sort()

### month wise check

In [ ]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd

# Convert min_date and max_date to Timestamp if they are date
min_date_ts = pd.to_datetime(min_date)
max_date_ts = pd.to_datetime(max_date)

print("Month-wise breakdown and missing dates:")

for month_start in months:
    month_start = pd.to_datetime(month_start)
    month_end = month_start + MonthEnd(0)
    # Dates existing in this month
    dates_in_month = check[check['date_only'].between(month_start.date(), month_end.date())]['date_only'].unique()
    dates_in_month = pd.to_datetime(dates_in_month)
    
    # Expected dates in month (from min to max, constrained by dataset range)
    expected_start = max(month_start, min_date_ts)
    expected_end = min(month_end, max_date_ts)
    expected_dates = pd.date_range(
        start=expected_start,
        end=expected_end
    )
    
    # Missing dates in this month
    missing_dates = expected_dates.difference(dates_in_month)
    
    print(f"{month_start.strftime('%B %Y')}:")
    print(f"  Dates present: {len(dates_in_month)}")
    print(f"  Dates missing: {len(missing_dates)}")
    if len(missing_dates) > 0:
        missing_str = ', '.join(missing_dates.strftime('%Y-%m-%d'))
        print(f"  Missing dates: {missing_str}")


### caluclation

In [ ]:
full_df_1 = check.copy()

In [ ]:
full_df_1['type'].unique()

In [ ]:
filtered_df = full_df_1[
    (~full_df_1['action'].isin(['Delete', 'Update'])) &
    (~full_df_1['type'].isin(['Pending', 'Withdraw Request', 'Deposit Request'])) &
    (~full_df_1['typeDetail'].isin(['SLTP', 'Adjustment']))
]

filtered_df['Date'] = pd.to_datetime(filtered_df['time']).dt.date
grouped = filtered_df.groupby(['accountId', 'Date'])

summary_df = grouped.apply(lambda g: pd.Series({
    'deposit_count': (g['typeDetail'] == 'Deposit').sum(),
    'deposit_amount': g.loc[g['typeDetail'] == 'Deposit', 'amount'].sum(),
    'withdrawal_count': (g['typeDetail'] == 'Withdrawal').sum(),
    'withdrawal_amount': g.loc[g['typeDetail'] == 'Withdrawal', 'amount'].sum(),
    'last_activity': g['time'].max(),
    'deposit_time': ''.join(
        f'[{i.strftime("%Y-%m-%d %H:%M:%S")}]' for i in sorted(g.loc[g['typeDetail'] == 'Deposit', 'time'])
    ),
    'deposit_distribution': ''.join(
        f'[{amt}]' for _, amt in sorted(zip(g.loc[g['typeDetail'] == 'Deposit', 'time'], g.loc[g['typeDetail'] == 'Deposit', 'amount']))
    )
})).reset_index()

In [ ]:
final = summary_df.copy()

In [ ]:
final.columns

In [ ]:

final['Date'] = pd.to_datetime(final['Date']).dt.date

for col in ['deposit_amount', 'withdrawal_amount']:
    final[col] = final[col].fillna(0).astype('int64')

final['last_activity'] = final['last_activity'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
cdb = final.copy()

In [ ]:
last_update_till = cdb['Date'].max()
last_update_till

In [ ]:
cdb['Date'] = pd.to_datetime(cdb['Date'])
unique_calls['Date'] = pd.to_datetime(unique_calls['Date'])

In [ ]:
unique_dates = unique_calls['Date'].dt.date.unique()
filtered_cdb = cdb[cdb['Date'].dt.date.isin(unique_dates)]

In [ ]:
unique_dates

In [ ]:
filtered_cdb

In [ ]:
import requests
import os
import json

# OAuth client details and refresh token
client_id = '1000.XYZABCDEF1234567890'
client_secret = 'your_client_secret_here'
refresh_token = '1000.abcdef1234567890abcdef1234567890.abcdef1234567890abcdef1234567890'

owner_email = 'ab@admin.com'
workspace_name = 'Zoho ky Analytics'

token_file = 'access_token.json'

def save_token(token):
    with open(token_file, 'w') as f:
        json.dump({'access_token': token}, f)

def load_token():
    if os.path.exists(token_file):
        with open(token_file, 'r') as f:
            data = json.load(f)
            return data.get('access_token')
    return None

def refresh_access_token():
    url = "https://accounts.zoho.com/oauth/v2/token"
    params = {
        'refresh_token': refresh_token,
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token'
    }
    response = requests.post(url, params=params)
    if response.status_code == 200:
        tokens = response.json()
        new_token = tokens['access_token']
        print("Refreshed Access Token:", new_token)
        save_token(new_token)
        return new_token
    else:
        raise Exception(f"Failed to refresh token: {response.text}")

def run_query(access_token, client_ids):
    # Convert all IDs to string (no lowercasing)
    client_ids_str = [str(x) for x in client_ids]
    id_list_str = "', '".join(client_ids_str)
    id_list_str = f"'{id_list_str}'"

    sql_query = f"""
        SELECT
            c."Account Id" as ID,
            f."Phone",
            c."Phone"
        FROM "Client" c
        LEFT JOIN "Followup" f ON f."Converted Contact" = c."Id"
        WHERE c."Account Id" IN ({id_list_str})
    """

    url = f"https://analyticsapi.zoho.com/api/{owner_email}/{workspace_name}"
    headers = {'Authorization': f"Zoho-oauthtoken {access_token}"}
    payload = {
        'ZOHO_ACTION': 'EXPORT',
        'ZOHO_OUTPUT_FORMAT': 'JSON',
        'ZOHO_SQLQUERY': sql_query,
        'ZOHO_API_VERSION': '1.0'
    }

    response = requests.post(url, headers=headers, data=payload)
    return response


# Usage Example:
client_ids = filtered_cdb['accountId'].unique().tolist()
#client_ids = ['825132', '824819', '825118']  # Example; replace with actual unique IDs

# Load token from storage or refresh if not present
access_token = load_token()
if not access_token:
    access_token = refresh_access_token()

response = run_query(access_token, client_ids)

if response.status_code == 401:
    print("Access token expired, refreshing...")
    access_token = refresh_access_token()
    response = run_query(access_token, client_ids)

if response.status_code == 200:
    data = response.json()
    print("Query Result:", data)
else:
    print("Error:", response.status_code, response.text)


# rows section above should not be null

In [ ]:
len(client_ids)

In [ ]:
columns = data['response']['result']['column_order']
rows = data['response']['result']['rows']
 
client_phone_1 = pd.DataFrame(rows, columns=columns)
client_phone_1.columns = ['Account Id', 'fphone','Phone']

In [ ]:
client_phone_1.nunique()

In [ ]:
client_phone_1 = client_phone_1.rename(columns={'Account Id' : "accountId", 'Phone' : 'f.Phone'})

In [ ]:
client_phone_1.columns

In [ ]:
def clean_phone(phone):
    if pd.isna(phone):
        return ""
    cleaned = ''.join(filter(str.isdigit, str(phone)))
    return cleaned[-10:]

In [ ]:
client_phone_1['f.Phone'] = client_phone_1['f.Phone'].apply(clean_phone)
client_phone_1['fphone'] = client_phone_1['fphone'].apply(clean_phone)

In [ ]:
# Function to validate the phone number based on the given conditions
def validate_phone(row):
    fphone = str(row['fphone'])
    fphone_check = str(row['f.Phone'])

    # Check if the fPhone is valid (i.e., it should have 10 digits)
    if len(fphone_check) != 10 or fphone_check[0] not in ['6', '7', '8', '9']:
        return fphone  # If invalid, return value from fphone column
    return fphone_check  # If valid, return fPhone value

# Apply the validation function
client_phone_1['ValidatedPhone'] = client_phone_1.apply(validate_phone, axis=1)

In [ ]:
client_phone_1 = client_phone_1 [['accountId','ValidatedPhone']]
client_phone_1 = client_phone_1.rename(columns = {'ValidatedPhone' : 'Consider'})

In [ ]:
client_manual = client_phone_1.copy()
client_manual['Consider'].replace('', np.nan, inplace=True)
client_manual.drop_duplicates(subset=['accountId'], inplace=True)
client_manual['Consider'].fillna('no phone found', inplace=True)
client_manual['accountId'] = client_manual['accountId'].astype(int)

In [ ]:
Dials_merged = unique_calls.merge(client_manual, on='Consider', how='left')
Deposits_merged = filtered_cdb.merge(client_manual, on='accountId', how='left')

In [ ]:
a = Dials_merged.merge(Deposits_merged[['accountId','deposit_amount']], on='accountId', how='left')
final = a.groupby(['CRM ID', 'Date']).agg({
    'accountId': 'nunique',   # counts unique accountId
    'deposit_amount': 'sum'
}).reset_index()

In [ ]:
final

In [ ]:
formatted_df


In [ ]:
df = formatted_df.merge(final, how='left', left_on=['CRM ID','Date'], right_on=['CRM ID','Date'])

## Export


In [ ]:
save_path = r'C:\Users\Akhil\Downloads\project\TradeX_report'
df.to_csv(f'{save_path}\\Summary_29_09.csv', index=False)
D.to_csv(f'{save_path}\\Dialer_29_09.csv', index=False)
crm_id_null_df.to_csv(f'{save_path}\\Not_Found_Users_29_09.csv', index=False)

In [ ]:
df